<img src="https://datascientest.fr/train/assets/logo_datascientest.png" style="height:150px"> 
<hr style="border-width:2px;border-color:##75DFC1">
<center> <h1> Projet COVID </h1> </center> <br>
<center> <h2> Premier prototype</h2> </center> <br>
<hr style="border-width:2px;border-color:##75DFC1">

Première implémentation d'un CNN sur les images
<hr style="border-width:2px;border-color:##75DFC1">

>CNN sur Keras

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

os.chdir('COVID-19 Radiography Database')

df = pd.read_csv('img_metadata.csv')

In [3]:
df.head()

,path,Class,height,width,channel,B&W,luminosite
0,COVID/COVID (934).png,covid,331.0,331.0,3.0,True,106.488376
1,COVID/COVID (296).png,covid,256.0,256.0,3.0,True,119.666504
2,COVID/COVID (74).png,covid,256.0,256.0,3.0,True,153.078995
3,COVID/COVID (261).png,covid,256.0,256.0,3.0,True,186.787781
4,COVID/COVID (978).png,covid,331.0,331.0,3.0,True,159.006818


In [4]:
print(df[['height', 'width']].value_counts())

height  width 
1024.0  1024.0    2686
331.0   331.0      573
256.0   256.0      558
891.0   1084.0      31
952.0   1184.0       9
992.0   1192.0       6
1047.0  1275.0       6
1053.0  1053.0       4
917.0   1116.0       4
852.0   1039.0       1
197.0   253.0        1
723.0   594.0        1
913.0   1102.0       1
875.0   1108.0       1
913.0   1108.0       1
928.0   1130.0       1
947.0   1192.0       1
160.0   187.0        1
dtype: int64


In [20]:
%matplotlib inline
import numpy as np

import matplotlib.pyplot as plt
import time, cv2
import seaborn as sns

import pandas as pd
from sklearn.model_selection import train_test_split

from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import Adam
from keras.models import Model, Sequential
from keras.preprocessing.image import ImageDataGenerator
import keras
from keras import backend as K

In [8]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale = 1./255)

In [12]:
data = df[['path', 'Class']]
data['Class'] = data['Class'].replace({'covid' : 1, 'normal' : 0, 'pneumo' : 2})

data_train, data_test = train_test_split(data, test_size=0.2, random_state=123)

<ipython-input-12-097d295e3a2f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Class'] = data['Class'].replace({'covid' : 1, 'normal' : 0, 'pneumo' : 2})


In [13]:
data_train

,path,Class
2396,NORMAL/NORMAL (1192).png,0
1120,COVID/COVID (865).png,1
1840,NORMAL/NORMAL (316).png,0
1134,COVID/COVID (219).png,1
1268,NORMAL/NORMAL (350).png,0
...,...,...
1122,COVID/COVID (1147).png,1
1346,NORMAL/NORMAL (193).png,0
3454,Viral Pneumonia/Viral Pneumonia (1096).png,2
3437,Viral Pneumonia/Viral Pneumonia (468).png,2


In [18]:
batch_size = 32

data_train["Class"] = data_train["Class"].astype(str)
data_test["Class"] = data_test["Class"].astype(str)

training_set = train_datagen.flow_from_dataframe(dataframe=data_train,
                                                 directory="",
                                                 x_col = "path",
                                                 y_col = "Class",
                                                 target_size = (160, 160), 
                                                 batch_size = batch_size, 
                                                 color_mode='grayscale',
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_dataframe(dataframe=data_test,
                                            directory="",
                                            x_col = "path",
                                            y_col = "Class",
                                            target_size = (160, 160),
                                            batch_size = batch_size,
                                            color_mode='grayscale',
                                            class_mode = 'categorical')

Found 3108 validated image filenames belonging to 3 classes.
Found 778 validated image filenames belonging to 3 classes.


<ipython-input-18-0ee7e16b7691>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train["Class"] = data_train["Class"].astype(str)
<ipython-input-18-0ee7e16b7691>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_test["Class"] = data_test["Class"].astype(str)


In [28]:
classifier = Sequential()
conv2d_5 = Conv2D(filters = 32, 
                  kernel_size = (3, 3), 
                  padding = 'valid', 
                  input_shape = (160, 160, 1), 
                  activation = 'relu')
max_pooling2d_5 = MaxPooling2D(pool_size = (2, 2))
conv2d_6 = Conv2D(filters = 32,                    
                  kernel_size = (3, 3),          
                  padding = 'valid',             
                  activation = 'relu')
max_pooling2d_6 = MaxPooling2D(pool_size = (2, 2))
flatten_3 = Flatten()
dense_5 = Dense(units = 128,
                activation = 'relu')
dense_6 = Dense(units = 3,
                activation = 'softmax')

classifier.add(conv2d_5)
classifier.add(max_pooling2d_5)
classifier.add(conv2d_6)
classifier.add(max_pooling2d_6)
classifier.add(flatten_3)
classifier.add(dense_5)
classifier.add(dense_6)

classifier.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 158, 158, 32)      320       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 79, 79, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 77, 77, 32)        9248      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 38, 38, 32)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 46208)             0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               5914752   
_________________________________________________________________
dense_7 (Dense)              (None, 3)                

In [29]:
# Compilation
classifier.compile(loss = 'categorical_crossentropy', 
                   optimizer = 'adam', 
                   metrics = ['accuracy'])

In [31]:
# Entraînement
history = classifier.fit(training_set, 
                                   epochs = 10,
                                   workers = -1,
                                   steps_per_epoch = len(data_train)//batch_size,
                                   validation_data = test_set,
                                   validation_steps = len(data_test)//batch_size)

Epoch 1/10
97/97 [==============================] - 47s 483ms/step - loss: 0.4689 - accuracy: 0.8257 - val_loss: 0.3427 - val_accuracy: 0.8815
Epoch 2/10
97/97 [==============================] - 46s 477ms/step - loss: 0.4811 - accuracy: 0.8023 - val_loss: 0.3574 - val_accuracy: 0.8672
Epoch 3/10
97/97 [==============================] - 46s 477ms/step - loss: 0.4114 - accuracy: 0.8384 - val_loss: 0.3022 - val_accuracy: 0.8932
Epoch 4/10
97/97 [==============================] - 47s 482ms/step - loss: 0.4039 - accuracy: 0.8457 - val_loss: 0.4054 - val_accuracy: 0.8542
Epoch 5/10
97/97 [==============================] - 46s 478ms/step - loss: 0.3513 - accuracy: 0.8774 - val_loss: 0.3013 - val_accuracy: 0.8906
Epoch 6/10
97/97 [==============================] - 47s 481ms/step - loss: 0.3959 - accuracy: 0.8492 - val_loss: 0.2950 - val_accuracy: 0.8932
Epoch 7/10
97/97 [==============================] - 48s 493ms/step - loss: 0.3694 - accuracy: 0.8605 - val_loss: 0.3989 - val_accuracy: 0.8503